## Selección de varaibles

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
loan_training_set4 = pd.read_csv('./data/01_raw/Loan_training_set_4_4.csv', sep = ',' , skiprows=1, skipfooter=2, engine = "python")
loan_training_set3 = pd.read_csv('./data/01_raw/Loan_training_set_3_4.csv', sep = ',' , skiprows=1, skipfooter=2, engine = "python")
loan_training_set2 = pd.read_csv('./data/01_raw/Loan_training_set_2_4.csv', sep = ',' , skiprows=1, skipfooter=2, engine = "python")
loan_training_set1 = pd.read_csv('./data/01_raw/Loan_training_set_1_4.csv', sep = ',' , skiprows=1, skipfooter=2, engine = "python")
frames = [loan_training_set1, loan_training_set2, loan_training_set3,loan_training_set4]
total_lendingclub = pd.concat(frames)

In [4]:
df = total_lendingclub.loc[:, loan_training_set1.columns.isin(["loan_status", "annual_inc",
"annual_inc_joint",
"application_type",
"dti",
"dti_joint",
"emp_length",
"grade",
"home_ownership",
"loan_amnt",
"mort_acc",
"open_act_il",
"open_rv_12m",
"pct_tl_nvr_dlq",
"pub_rec",
"pub_rec_bankruptcies",
"purpose",
"term",
"tot_coll_amt"
"tot_cur_bal",
"total_acc",
"int_rate"
])]


Para los valores dentro de nuestra variable `loan_status` quitamos todos aquellos que sean ed tipo current.

In [5]:
df = df[df["loan_status"]!="Current"]

### Variables con NA

Como podemos observa, existen varias variables que contienen un gran numero de NA's. Se ha decidido que nos resultan relevantes estas variables y por lo tanto se usarán para la creación de modelos ya que las variables con alto número de NA's corresponden a préstamos conjuntos cuando el préstamo es individual. 

In [6]:
df.isnull().sum()

loan_amnt                    0
term                         0
int_rate                     0
grade                        0
emp_length               25086
home_ownership               0
annual_inc                   0
loan_status                  0
purpose                      0
dti                         21
pub_rec                      0
total_acc                    0
application_type             0
annual_inc_joint        460541
dti_joint               460541
open_act_il             430839
open_rv_12m             430839
mort_acc                     0
pct_tl_nvr_dlq               0
pub_rec_bankruptcies         0
dtype: int64

## Variables numéricas

En primer lugar, se procede a convertir una de las variables que se considera relevantes para hacer los modelos como es el tipo de interés. Para ello, se elimna el símbolo de % que hace que sea de tipo string y de seguido lo convertimos a tipo entero.  

In [7]:
int_numeric = df["int_rate"].str.strip("%")
int_numeric = pd.to_numeric(int_numeric)
df["int_rate"] = int_numeric

A continuación, se seleccionan todas aquellas variables del dataframe que son numéricas utilizando un pipeline con SimpleImputer con la mediana para los valores perdidos y se procede a escalar los números.

In [12]:
numericas = df.select_dtypes(exclude=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

Aquí podemos observar todas las variables numéricas

In [13]:
numericas_df = df.select_dtypes(exclude=['object'])

In [14]:
numericas_df

,loan_amnt,int_rate,annual_inc,dti,pub_rec,total_acc,annual_inc_joint,dti_joint,open_act_il,open_rv_12m,mort_acc,pct_tl_nvr_dlq,pub_rec_bankruptcies
8,11575,7.35,153000.0,16.99,1,46,NaN,NaN,1.0,2.0,2,95.7,1
10,7200,24.85,50000.0,6.07,0,5,NaN,NaN,1.0,0.0,0,80.0,0
24,7500,7.35,110000.0,13.12,0,27,NaN,NaN,4.0,1.0,4,100.0,0
42,10000,16.02,51979.0,10.11,2,21,NaN,NaN,1.0,2.0,0,100.0,2
91,14000,16.02,75000.0,10.86,0,10,NaN,NaN,2.0,2.0,1,90.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
421088,11575,15.59,25400.0,27.08,0,18,NaN,NaN,NaN,NaN,0,100.0,0
421089,12000,11.99,63000.0,23.69,0,61,NaN,NaN,NaN,NaN,1,98.3,0
421092,13000,15.99,35000.0,30.90,1,22,NaN,NaN,NaN,NaN,0,100.0,1
421093,12000,19.99,64400.0,27.19,0,20,NaN,NaN,NaN,NaN,0,95.0,0


## Variables Categóricas

A continuación realizamos la misma transformación para las variables categóricas.

In [15]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [16]:
categoricas = df.select_dtypes(include=['object']).drop(['loan_status'], axis=1).columns

categoricas

Index(['term', 'grade', 'emp_length', 'home_ownership', 'purpose',
       'application_type'],
      dtype='object')

In [17]:
categoricas_df = df.select_dtypes(include=['object']).drop(['loan_status'], axis=1)

In [18]:
categoricas_df

,term,grade,emp_length,home_ownership,purpose,application_type
8,36 months,A,6 years,OWN,credit_card,Individual
10,36 months,E,2 years,RENT,debt_consolidation,Individual
24,36 months,A,7 years,MORTGAGE,debt_consolidation,Individual
42,60 months,C,7 years,RENT,debt_consolidation,Individual
91,36 months,C,7 years,MORTGAGE,debt_consolidation,Individual
...,...,...,...,...,...,...
421088,36 months,D,10+ years,RENT,credit_card,Individual
421089,36 months,B,< 1 year,MORTGAGE,credit_card,Individual
421092,60 months,D,5 years,RENT,debt_consolidation,Individual
421093,60 months,E,1 year,RENT,debt_consolidation,Individual


## Concatenación

Se crea un dataframe de la variable target.

In [19]:
loan_status = pd.DataFrame(df['loan_status'])

Se concatenan tanto la variable target como las variables numéricas y categóricas ya transformadas.

In [20]:
lendingclub = pd.concat([numericas_df, loan_status, categoricas_df],  axis=1)

Se realiza un pipeline con las variables categóricas para más tarde emplear en los modelos.

In [21]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numericas),
        ('cat', categorical_transformer, categoricas)])

## Variable Target

Vemos que en nuestra variable target tenemos 7 valores posibles, para este trabajo haremos un encode entre las que sean Fully Paid y el resto de variables. Por lo tanto las que sean Fully Paid tendrán como valor el 1 y el resto tomarán el valor 0.

In [22]:
lendingclub["loan_status"].value_counts()

Fully Paid            345520
Charged Off            97047
Late (31-120 days)     11168
In Grace Period         5507
Late (16-30 days)       2915
Default                   34
Name: loan_status, dtype: int64

In [23]:
lendingclub['loan_status'].mask(lendingclub['loan_status'] == 'Fully Paid', 1, inplace=True)
lendingclub['loan_status'].mask(lendingclub['loan_status'] != 1, 0, inplace=True)
lendingclub

,loan_amnt,int_rate,annual_inc,dti,pub_rec,total_acc,annual_inc_joint,dti_joint,open_act_il,open_rv_12m,mort_acc,pct_tl_nvr_dlq,pub_rec_bankruptcies,loan_status,term,grade,emp_length,home_ownership,purpose,application_type
8,11575,7.35,153000.0,16.99,1,46,NaN,NaN,1.0,2.0,2,95.7,1,1,36 months,A,6 years,OWN,credit_card,Individual
10,7200,24.85,50000.0,6.07,0,5,NaN,NaN,1.0,0.0,0,80.0,0,1,36 months,E,2 years,RENT,debt_consolidation,Individual
24,7500,7.35,110000.0,13.12,0,27,NaN,NaN,4.0,1.0,4,100.0,0,1,36 months,A,7 years,MORTGAGE,debt_consolidation,Individual
42,10000,16.02,51979.0,10.11,2,21,NaN,NaN,1.0,2.0,0,100.0,2,1,60 months,C,7 years,RENT,debt_consolidation,Individual
91,14000,16.02,75000.0,10.86,0,10,NaN,NaN,2.0,2.0,1,90.0,0,1,36 months,C,7 years,MORTGAGE,debt_consolidation,Individual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421088,11575,15.59,25400.0,27.08,0,18,NaN,NaN,NaN,NaN,0,100.0,0,1,36 months,D,10+ years,RENT,credit_card,Individual
421089,12000,11.99,63000.0,23.69,0,61,NaN,NaN,NaN,NaN,1,98.3,0,0,36 months,B,< 1 year,MORTGAGE,credit_card,Individual
421092,13000,15.99,35000.0,30.90,1,22,NaN,NaN,NaN,NaN,0,100.0,1,0,60 months,D,5 years,RENT,debt_consolidation,Individual
421093,12000,19.99,64400.0,27.19,0,20,NaN,NaN,NaN,NaN,0,95.0,0,0,60 months,E,1 year,RENT,debt_consolidation,Individual


## Dataset final para los modelos

In [ ]:
lendingclub.to_csv('./data/03_processed.csv', index = False)